# Exposing class inheritance and attribute characteristics with namedtuple()

So, this is an attempt to reveal to you some of the subtle differences between regular classes in Python and special classes.

The enquiry into these differences started with the difference between mutable and immutable types in Python, an early learning subject most beginners are familiar with.

We know everything in Python is an object.

But not every object is alike.

True fact: namedtuple() is a function of the collections library, so it can be called a method.

So, another reason to look at this cropped up in a learning splash into Descriptors. There, to save on having to repeat _get, _set, and _del methods to handle update requests on instances objects, and to provide specialized functionality such as data validation before an operation with such a handler commits, Descriptors step in to shave back the need for unweildy repetition.

The attempt here is to extend namedtuple class. One goal is to find out if it's possible to implement property validation on the class.

We'll look at some crafty ABC.meta metaclass code which shows off a little bit about Descriptors extending by offset the native _get, _set, _del, and _new methods of a class. These can be exposed to the baseclass as its metaclass properties, but whether namedtuple will allow chenanigans of type or two is not yet known.

Let's gear up and dive in.

In [1]:
from collections import namedtuple as nt

attribs = ['name', 'rank', 'suit', 'increment_val','init_val', 'current_val', 'modifier_val']
vals = (None, 'jack', 'clubs', 1, 0, 0, 1) # will come from a generated dict, for now a single demonstration class

First, some incorrect syntax, to show we can't explicitly reference inheritable classes, or the properties of the namedtuple baseclass using the namedtuple() function.

In [2]:
Card(object) = nt('Card', attribs)

SyntaxError: can't assign to function call (<ipython-input-2-3a35d498899d>, line 1)

Similarly, the reference to an inheritable class can't be made where the name of the new class is established here.

In [3]:
Card = nt('Card(object)', attribs)

ValueError: Type names and field names must be valid identifiers: 'Card(object)'

The valid syntax below works fine, and we go on beneath to instance a card from class Card.

In [5]:
Card = nt('Card', attribs)

In [6]:
card = Card(*vals)

Inspect the class to determine what values can be set on the base class prior to instance construction(s).


In [7]:
dir(dir(Card))

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [8]:
dir(Card.__init_subclass__())

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Now, we'll turn to some deep dive takes on Descriptors, in particular a pattern which makes use of metaclass properties of a class to implement a callback.
[Descriptor Writeup - Chris Beaumont](http://nbviewer.jupyter.org/gist/ChrisBeaumont/5758381/descriptor_writeup.ipynb)

In [9]:
class Descriptor(object):
    def __init__(self):
        #notice we aren't setting the label here
        self.label = None
        
    def __get__(self, instance, owner):
        print('__get__. Label = %s' % self.label)
        return instance.__dict__.get(self.label, None)
    
    def __set__(self, instance, value):
        print('__set__')
        instance.__dict__[self.label] = value

        
class DescriptorOwner(type):
    print('Found class DecriptorOwner')
    def __new__(cls, name, bases, attrs):
        # find all descriptors, auto-set their labels
        for n, v in attrs.items():
            if isinstance(v, Descriptor):
                v.label = n
        return super(DescriptorOwner, cls).__new__(cls, name, bases, attrs)
    
class Constructor():
    ''' '''
    def __init__(self):
        print(self, 'There will be no initialization to speak of.')
     
class Foo(object):
    __metaclass__ = DescriptorOwner

Found class DecriptorOwner


Before we go on to refactor the code to fit, we need to find out if the dunder metaclass property of class Foo shown here is also a property of namedtuple class Card. We've seen so far by inspection using dir() that the property doesn't appear to exist on the class, and that may mean our attempt here is going to hit a brick wall.

Let's reuse the lists from above to populate the class attributes and instance values, but change the name of the class we want to create, to avoid confusion with the above.

The next line of code is taken from the example above in which class Foo: is shown to have a class variable which sets the __metaclass__ attribute to the class DescriptorOwner, an object somewhat more than a handler which, well, handles new class construction requests, checking to see if the name of the property exists in its dictionary of labeled attributes. It does this to get around limitations of the Descriptor framework which prevent list class property references from being properly saved so Descriptor handlers can be engaged, treating all instance variables,  regardless of their affinity with Descriptors, the same -- by labeling them as the name of the variable to which a given call refers, making the map (dictionary) of references available to route requests as appropriate. 

The methods exposed in the metaclass handle calls made to standard class methods, methods intrinsic to all normal classes. 

In [10]:
Banner = nt('Banner', attribs)
Banner.__metaclass__ = DescriptorOwner
# TODO []
# Banner.__init_subclass__ =  Constructor  # DescriptorOwner._init_here()
# Banner.__init_subclass__ =
dir(Banner)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 '_source',
 'count',
 'current_val',
 'increment_val',
 'index',
 'init_val',
 'modifier_val',
 'name',
 'rank',
 'suit']

Well, that seems to work after all.
Now let's make an instance, and check out a few relevant things about it.

In [11]:
b = Banner(*vals)
print(b.rank)

jack


In [12]:
b.rank = 'queen'
print(b.rank)

AttributeError: can't set attribute

As it turns out, we have hit a final snag on the quest to validate new values assigned to named tuple instanced objects. If you don't see an AttributeError above after running all code above, then it's a miracle.

By its nature, a tuple is immutable, meaning that its values can't be changed on the instance, or in the baseclass once they have been created. The only workaround is not really a workaround at all, that is, to create a new instance whose values are a replica of another instance, changing those we want prior.
It may seem a little strange, but printing the values along the way should help disambiguate which values stick around and on whom.

##### namedtuple _replace() method

The semi-private _replace() method doesn't quite work the way we might expect from its name, though it does facilitate a blueprint from a populated instance.

First, the part that doesn't work.

In [13]:
# b.rank = 'queen'
b._replace(rank='queen')
print(b)

Banner(name=None, rank='jack', suit='clubs', increment_val=1, init_val=0, current_val=0, modifier_val=1)


Alright, earlier we saw setting the value on an existing instance doesn't work, and the above confirms this.
Now for the bit that brings inheritance into question while making short work of a new instance.

In [14]:
e = b._replace(rank='king', suit='hearts')
print(e)

Banner(name=None, rank='king', suit='hearts', increment_val=1, init_val=0, current_val=0, modifier_val=1)


The burning question: Does the newly instanced object become a subclass to its sibling?

In [15]:
# print(isinstance(Banner, (e, b))
print(type(b))
print(type(e))
print(type(Banner))
print(isinstance(Banner, type(b)))
print(isinstance(Banner, type(e)))
print(isinstance(b, type(e)))
print(isinstance(e, type(b)))

<class '__main__.Banner'>
<class '__main__.Banner'>
<class 'type'>
False
False
True
True


In [16]:
f = Banner._replace(rank='king', suit='hearts')
print(f)

TypeError: _replace() missing 1 required positional argument: '_self'

Clearly, the baseclass won't accept replacement values, and by the rules of inheritance as implemented in most language, Python included, instanced classes don't create new objects of their parent's type -- unless, for some reason, they're designed to do so.

If you'd like to see the actual code for namedtuple class there are several ways to do so.
For the simplest, run the following in your terminal:

`pydoc3.6 -p 4200`

Then navigate to:

`http://localhost:4200/getfile?key=/usr/lib/python3.6/collections/__init__.py`

And use Ctrl-F to find: 

`class {typename}(tuple):`

### Descriptor wrap up

Any remaining work on descriptors will have to use a simple class whose derivation is not limited by the named tuple base class.

In some sense, leaving Descriptors here is a bit of a let down.

However, one might call what remains from this enquiry a major advantage.

How so, you ask? Well, we get security.

Using the namedtuple method, new values may only be instanced, so each object off the baseclass is essentially read only after being instanced. That simple property of tuples extends securely to objects.

We have seen, however, that not all is read only. It is possible to add attributes to the baseclass following its creation that do end up on all instanced classes without regard to when those attributes are added, as it should be for class variables, always available instantly, to object instances with access to baseclass variables and methods.

In [17]:
print(Banner.__metaclass__)

<class '__main__.DescriptorOwner'>


In [18]:
print(e.__metaclass__)

<class '__main__.DescriptorOwner'>


As it turns out, we can set properties on the base class that can be inherited. Logic seems to dictate that for such properties to be available to concrete classes they must be set before an instance is created.

Following this logic, let's see if we can also set a new property of one of our instanced classes that none of the other instance of the same parent class have.

In [19]:
e._flamewar = 'hellfire'
print(e.flameware)

AttributeError: 'Banner' object has no attribute '_flamewar'

So, the answer is a resounding 'No'.

Let's set the property on the parent class.
Remember, it was supposed properties set on the baseclass would only be available to instances if the property was there when they were instanced. 
The rest shows how that take on baseclass and instance behavior and access is wrong.
As long as the property on the parent is set and exposed when called for by an instance, it's there on the instance prefixed reference to the attribute. 

In [20]:
Banner.flamewar = 'fire_hydrant'
print(Banner.flamewar)

fire_hydrant


In [21]:
print(e.flamewar)

fire_hydrant


In [22]:
print(b.flamewar)

fire_hydrant


Alright, now we know.

To be sure you know because you've seen the right output, please review the errors in the code above. They're there to reveal pitfalls. The interpreter will stop, letting you read and click to run the code below in sequence.

You should see both e.flamewar and b.flamewar attributes linked to the baseclass, showing the value 'fire_hydrant'.

Restart to refresh your kernel if necessary.

# Generate a deck of cards

For clarity, let's import namedtuple from collection again. And we'll set up a class like earlier, instancing one to get at replicable default values.

In [ ]:
from collections import namedtuple as nt

ranks = {'2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, 'jack': 10, 'queen': 11, 'king': 12, 'ace': 13}
suits = ['spades', 'hearts', 'clubs', 'diamonds']
attribs = ['name', 'rank', 'suit', 'val_init', 'val_current', 'isboss', 'isshown', 'isheld', 'holder']
protovals = None, None, None, None, None, False, False, False, None
cardlist = []
Chance = nt('Chance', attribs)
proto = Chance(*protovals)
print(proto)

for s in suits:
    for r, v in ranks.items():
        cardlist.append(proto._replace(name=f'{r} of {s}', rank=r, suit=s, val_init=v))

for _ in cardlist:
    print(_)

### Run and choose, run and choose, run and choose
Reading this in Jupyter Notebook you and me both can benefit from the cellular interconnectedness and isolation. No while statement needs to be present to let you click to fire off one card choice after another, so none has to be written.

In [ ]:
 
from random import choice

print(choice(cardlist))
         

# Next on my agenda

Some of the next items on my agenda are ones with learning curves which still give me a lot of grief, so while  learn more about themand up for grabs.

1. This series speaks to the ABC.meta methods:

    __subclasscheck__
    __subclasshook__
    issubclass()
    
One of the best resources I've found so far, besides the PEP, is [stackoverflow: python-subclasscheck-subclasshook](https://stackoverflow.com/questions/40764347/python-subclasscheck-subclasshook).

2. Several other properties of classes of great interest as seen above under the microscope with dir().

    __class__
    __init__
    __init__subclass__

1. And finally, looking at `import inspect`, `super()`, and finding and learning about other contributors in the language to an understanding of the MRO (Method Resolution Order) and metaclass patterns.

Fun times ahead!

I'm in the process of learning Python, and this file is reflective of that, it being a work in progress as well.

So, I do appreciate my use of language on aspects of the Python subject may not be perfect, or as easy to follow as it might be coming from someone who has crunched complex concepts into simple and erudite morsels, which we all know make it easiest to learn and retain important points.

Please, let me know if you have any questions, comments or... revisions.

Reach me, Tim, rebelclause@gmail.com.

July 27, 2018